In [ ]:
!pip install surprise
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
import pandas as pd
import numpy as np
import json
from surprise import SVD,SVDpp,NMF,SlopeOne,KNNBasic,KNNBaseline,CoClustering

     |████████████████████████████████| 11.8MB 292kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617538 sha256=489317f4643d5891227006483e7ca6468b79d1564d12a63d86cf761005b8ef7f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Loading train data into a dataframe
train_dataframe = pd.read_csv('/content/gdrive/My Drive/train.csv')[['userId', 'movieId', 'rating']]
train_dataframe.head()

,userId,movieId,rating
0,1,2692,4
1,1,1566,4
2,1,2762,4
3,1,588,4
4,1,938,4


In [ ]:
reader = Reader(rating_scale=(1, 5))
training_data = Dataset.load_from_df(train_dataframe, reader).build_full_trainset()

In [ ]:
# Loading test data into a dataframe
test_dataframe = pd.read_csv('/content/gdrive/My Drive/test.csv')[['userId', 'movieId', 'rating']]
test_dataframe.head()

,userId,movieId,rating
0,1,1193,5
1,1,3408,4
2,1,919,4
3,1,2797,4
4,1,720,3


In [ ]:
#algorithm = SVD()
#algo = 'SVD'
#algorithm = NMF()
#algo = 'NMF'
#algorithm = CoClustering()
#algo = 'CoClustering'
algorithm = KNNBaseline()
algo = 'KNNBaseline'

In [ ]:
# training the model with the training data
algorithm.fit(training_data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
#function to predict ratings with the help of trained model
attributes_predict = []
def predict_ratings(data):
  predict_list = []
  prediction = algorithm.predict(data[0], data[1])
  predict_list.append(int(prediction[0]))
  predict_list.append(int(prediction[1]))
  predict_list.append(prediction[3])
  attributes_predict.append(predict_list)

#testing the trained model
test_dataframe.apply(predict_ratings, axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
200011    None
200012    None
200013    None
200014    None
200015    None
Length: 200016, dtype: object

In [ ]:
# combining predicted ratings with the true ratings
result_dataframe = pd.DataFrame.from_records(attributes_predict, columns=['userId', 'movieId', 'predicted_rating'])
result_dataframe['true_rating'] = test_dataframe['rating']
result_dataframe.head()

,userId,movieId,predicted_rating,true_rating
0,1,1193,4.375000,5
1,1,3408,4.025000,4
2,1,919,4.476565,4
3,1,2797,4.096597,4
4,1,720,4.525572,3


In [ ]:
# saving resulted predicted ratings into a csv file
result_dataframe.to_csv('/content/gdrive/My Drive/' + algo + '.csv', index=False)